<a href="https://colab.research.google.com/github/matrix007-creator/car_catalogue/blob/master/predicting_mpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install tensorflow==2.0.0-beta1


  Using cached https://files.pythonhosted.org/packages/29/6c/2c9a5c4d095c63c2fb37d20def0e4f92685f7aee9243d6aae25862694fd1/tensorflow-2.0.0b1-cp36-cp36m-manylinux1_x86_64.whl
     |████████████████████████████████| 3.1MB 3.3MB/s 
  Using cached https://files.pythonhosted.org/packages/32/dd/99c47dd007dcf10d63fd895611b063732646f23059c618a373e85019eb0e/tf_estimator_nightly-1.14.0.dev2019060501-py2.py3-none-any.whl
     |████████████████████████████████| 51kB 6.4MB/s 
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [1]:
!pip install seaborn

In [3]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import pathlib
import matplotlib.pyplot as plt
from tensorflow import keras

print(tf.__version__)

2.0.0-beta1


In [4]:
data_path=keras.utils.get_file('auto-mpg.data',"https://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
data_path

'/root/.keras/datasets/auto-mpg.data'

In [5]:
column_names=['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration','Model Year','Origin']
r_dataset=pd.read_csv(data_path,names=column_names,na_values='?',comment='\t',sep=" ",skipinitialspace=True)
dataset=r_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [6]:
dataset.isna().sum()

MPG             0
Cylinders       0
Displacement    0
Horsepower      6
Weight          0
Acceleration    0
Model Year      0
Origin          0
dtype: int64

In [7]:
dataset=dataset.dropna()

In [8]:
origin=dataset.pop('Origin')

In [9]:

dataset['USA']=(origin==1)*1
dataset['Europe']=(origin==2)*1
dataset['Japan']=(origin==3)*1

In [10]:
train_ds=dataset.sample(frac=0.8,random_state=0)
test_ds=dataset.drop(train_ds.index)

In [11]:
train_stats=train_ds.describe()
train_stats.pop("MPG")
train_stats=train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


In [12]:
train_lbl=train_ds.pop("MPG")
test_lbl=test_ds.pop("MPG")

In [13]:
def norm(x):
  return (x-train_stats['mean'])/train_stats['std']
norm_train_data=norm(train_ds)
norm_test_data=norm(test_ds)

In [16]:
def build_model():
  model=keras.Sequential([
                          keras.layers.Dense(64,activation='relu',input_shape=[len(train_ds.keys())]),
                          keras.layers.Dense(64,activation='relu'),
                          keras.layers.Dense(1)
  ])
  optimizer=tf.keras.optimizers.RMSprop(0.001)
  model.compile(
      loss='mse',
      optimizer=optimizer,
      metrics=['mae','mse']
  )
  return model

In [18]:
model=build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                640       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________
